In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import where
import imblearn
print(imblearn.__version__)

data = pd.read_csv('/kaggle/input/health-insurance-cross-sell-prediction/train.csv')
print(data.head(2))
X = data[data.columns.drop('Response')]
y = data['Response']

X=pd.DataFrame(X)
y=pd.DataFrame(y)

X.loc[X['Gender'] == 'Male', 'Gender'] = 1
X.loc[X['Gender'] == 'Female', 'Gender'] = 0

X.loc[X['Vehicle_Age'] == '> 2 Years', 'Vehicle_Age'] = 2
X.loc[X['Vehicle_Age'] == '1-2 Year', 'Vehicle_Age'] = 1
X.loc[X['Vehicle_Age'] == '< 1 Year', 'Vehicle_Age'] = 0

X.loc[X['Vehicle_Damage'] == 'Yes', 'Vehicle_Damage'] = 1
X.loc[X['Vehicle_Damage'] == 'No', 'Vehicle_Damage'] = 0

feature_cols = X.keys()
print(X.keys())

## transform the dataset
#oversample = SMOTE()
#X, y = oversample.fit_resample(X, y)

# define pipeline
over = SMOTE(sampling_strategy='minority')
under = RandomUnderSampler()
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)
# transform the dataset
X, y = pipeline.fit_resample(X, y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test
# Create Decision Tree classifer object
clf = DecisionTreeClassifier(criterion="entropy", max_depth=3)

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
from sklearn.metrics import classification_report

print (classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test, y_pred)

In [ ]:
test=pd.read_csv('/kaggle/input/health-insurance-cross-sell-prediction/test.csv')
print(test.head(2))
X1 = test

X1=pd.DataFrame(X1)

X1.loc[X1['Gender'] == 'Male', 'Gender'] = 1
X1.loc[X1['Gender'] == 'Female', 'Gender'] = 0

X1.loc[X1['Vehicle_Age'] == '> 2 Years', 'Vehicle_Age'] = 2
X1.loc[X1['Vehicle_Age'] == '1-2 Year', 'Vehicle_Age'] = 1
X1.loc[X1['Vehicle_Age'] == '< 1 Year', 'Vehicle_Age'] = 0

X1.loc[X1['Vehicle_Damage'] == 'Yes', 'Vehicle_Damage'] = 1
X1.loc[X1['Vehicle_Damage'] == 'No', 'Vehicle_Damage'] = 0

feature_cols = X1.keys()
print(X1.keys())

In [ ]:
id=test.id
Preds = [pred[1] for pred in clf.predict_proba(X1)]
submission = pd.DataFrame(data = {'id': id, 'Response': Preds})
submission.to_csv('vehicle_insurance.csv', index = False)
submission.head()